In [ ]:
!pip install matplotlib

In [1]:
#%%
from Modele_decodeur_maison import *

from Pipeline import *
from Trainer import * 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 50

# Images
# images = np.load("C:/Users/lucas/Desktop/train-resnet50-res4frelu.npy")

# Texts
tokenized_fr,tokenized_en, vocab_fr,vocab_en = get_train_data_nouveau(batch_size)
n_token_fr = len(vocab_fr.keys())
n_token_en = len(vocab_en.keys())

inv_map_en = {v: k for k, v in vocab_en.items()}
inv_map_fr = {v: k for k, v in vocab_fr.items()}

n_head = 4 
num_encoder_layers = 2
num_decoder_layers = 2
dim_feedforward = 196
dropout = 0.1
activation = nn.Softmax
embedding_dim = 196

model_fr = Modèle(n_token_fr,embedding_dim,n_head, num_encoder_layers,num_decoder_layers,dim_feedforward,dropout,activation).to(device)

model_en = Modèle(n_token_en,embedding_dim,n_head, num_encoder_layers,num_decoder_layers,dim_feedforward,dropout,activation).to(device)





DEBUT_DE_PHRASE
FIN_DE_PHRASE
TOKEN_VIDE
&@@
j
;@@
DEBUT_DE_PHRASE
FIN_DE_PHRASE
TOKEN_VIDE
&@@
j
ë
ë@@


In [ ]:

# Y=train_auto_encoding(model_fr,tokenized_fr)
# Y=train_auto_encoding(model_en,tokenized_en)
# Y= cycle_consistency_train(model_fr,model_en,tokenized_fr,tokenized_en)
# mixed_train(model_fr,model_en,tokenized_fr,tokenized_en,200,50)



In [2]:
#%%
def tensor_to_sentence(output,inv_dic):
    result = [inv_dic[int(x)] for x in output]
    sentence = ""
    for word in result : 
        if word == "DEBUT_DE_PHRASE" :
            pass
        elif '@@' in word: 
            sentence+=word[:-2]
        elif word == "FIN_DE_PHRASE" :
            break 
        else :
            sentence+=word +" "
    return sentence

def traduit(model_A,model_B,data, inv_map):
    model_en.eval()
    model_fr.eval()
    output = torch.argmax(cycle_consistent_forward(model_A,model_B,data),dim = 2).T[j].T
    return tensor_to_sentence(output.view(-1),inv_map)

i = np.random.randint(len(tokenized_en)//50)
j = np.random.randint(50)
data,target = get_batch(tokenized_en, i)

print("Phrase à traduire : \n" + tensor_to_sentence(target.T[j],inv_map_en)+ "\n Phrase traduite : \n "+ traduit(model_en,model_fr,data, inv_map_fr))


#%%

Phrase à traduire : 
a happy young boy sits in a chair with a large elmo doll . 
 Phrase traduite : 
 casino attend es ongles chevauche équipmoyen tirent perron argile moyen soigneusement antiperron attend perron es perron planche tranquille ongles mières fluoreannperron polo médecins médecins médecins médecins médecins solo envotwister twister médecins médecins médecins médecins èque èque èque èque èque èque èque médecins médecins médecins médecins médecins las las las médecins médecins médecins médecins envoenvoenvoenvoestomac estomac estomac envoenvogueugueumédecins médecins médecins estomac estomac estomac estomac estomac estomac las las gueugueugueugueuèque èque tronc tronc tronc tronc tronc envoenvoenvoenvoenvoéquilibre 


/tmp/ipykernel_8621/3639088073.py:19: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3277.)
  output = torch.argmax(cycle_consistent_forward(model_A,model_B,data),dim = 2).T[j].T


In [2]:
import os

import s3fs
import pandas
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

val_features  = np.load(fs.open('tordjx/val-resnet50-res4frelu.npy'))
# train_features  = np.load(fs.open('tordjx/train-resnet50-res4frelu.npy'))


In [3]:
data_text, _  = get_batch(tokenized_fr, 0)
batchsz = 50
embed = model_fr.embedding(data_text)
torch.cat((embed, torch.from_numpy(val_features.reshape(1014,1024,196)[:1000].reshape(1000//batchsz ,1024,batchsz , 196)[0]).to(device))).shape

torch.Size([1121, 50, 196])

In [5]:
output = model_fr(data_text, True, torch.from_numpy(val_features.reshape(1014,1024,196)[:1000].reshape(1000//batchsz ,1024,batchsz , 196)[0]).to(device= device, dtype = torch.float32)).shape

torch.Size([97, 50, 196]) torch.Size([1024, 50, 196])
tensor([[[ 0.7049,  0.8399,  0.9887,  ..., -2.1492, -0.0053,  1.8226],
         [ 0.3827, -0.1423,  0.4637,  ..., -1.8663, -0.3090,  1.6264],
         [ 0.4837,  0.7676,  0.4224,  ..., -1.3742, -0.2670,  1.6772],
         ...,
         [ 0.4163,  0.6139,  0.7413,  ..., -1.8274, -0.2884,  1.7489],
         [-0.2638,  1.0282,  1.0514,  ..., -1.6033,  0.3842,  2.4883],
         [-0.2210,  0.9907,  0.6721,  ..., -1.7667,  0.4140,  1.7915]],

        [[-0.4685,  0.7395,  0.2914,  ...,  0.1102,  0.2189, -0.8870],
         [-0.4287,  0.5639, -0.9865,  ..., -1.0535,  0.5909, -0.6768],
         [-1.0402,  0.3018,  1.2465,  ..., -0.6023,  0.4301,  0.8055],
         ...,
         [-0.7018, -0.4500,  2.2858,  ..., -0.3024, -0.0880,  0.5852],
         [-0.1163, -0.3267,  0.3846,  ..., -2.8438, -0.1126, -0.2534],
         [-0.6118,  0.1725,  2.8546,  ...,  0.5841,  0.3889,  0.7734]],

        [[-1.6710,  0.1333,  0.6920,  ...,  0.5399,  0.3211,  

TypeError: can only concatenate list (not "Tensor") to list